In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
df_met = pd.read_csv('../data/xls_Bulath/Bullath_metadata.csv',sep = '\t')
df_smb      = pd.read_excel('../data/xls_Bulath/Bullath_all.xls', header=None)
smb_data    = df_smb.values[:,1:-1]
smb_time    = df_smb.values[:,0]

In [3]:
df_smb.columns = df_met.Name_orig

In [4]:
smb_coll = []
for i in df_met.Name_orig[1:]:
    subset = (df_smb[['Date', i]])
    subset.rename(columns = {i:'SMB'},  inplace=True)
    subset['Stake'] = i
    g_met = df_met.loc[lambda df_met: df_met['Name_orig'] == i, :]
    subset['Latitude']  = float(g_met.Latitude.values)
    subset['Longitude'] = float(g_met.Longitude.values)
    subset['Elevation'] = float(g_met.Elevation.values)
    subset.dropna(inplace=True)
    date_2 = []
    date_2.append(subset.Date[subset.index[0]])
    for j in subset.Date[0:-1]:
        date_2.append(j)

    smb_2 = []
    smb_2.append(0)
    for j in subset.SMB[0:-1]:
        smb_2.append(j)

    subset['Date1'] = date_2
    subset['SMB1'] = smb_2
    subset['Days'] = (subset['Date'] - subset['Date1']).dt.days
    subset['SMB_diff'] = -(subset['SMB1'] - subset['SMB'])
    subset = subset[['Stake', 'Date', 'Date1','Latitude', 'Longitude',
                     'Elevation', 'Days', 'SMB', 'SMB1', 'SMB_diff']]
    smb_coll.append(subset)

In [5]:
smb_coll_df = pd.concat(smb_coll, ignore_index=True)
smb_coll_df[['SMB_diff', 'SMB']] = smb_coll_df[['SMB_diff', 'SMB']]/100

In [6]:
df_s = smb_coll_df.sort_values(by=['Elevation', 'Date'])

In [7]:
df_s

Name_orig,Stake,Date,Date1,Latitude,Longitude,Elevation,Days,SMB,SMB1,SMB_diff
1397,R15,2008-01-05,2008-01-05,-62.178873,-58.907921,54.0,0,0.49203,0.000,0.49203
1398,R15,2008-01-26,2008-01-05,-62.178873,-58.907921,54.0,21,0.26200,49.203,-0.23003
1399,R15,2008-02-02,2008-01-26,-62.178873,-58.907921,54.0,7,0.23310,26.200,-0.02890
1400,R15,2008-02-09,2008-02-02,-62.178873,-58.907921,54.0,7,0.18954,23.310,-0.04356
1401,R15,2008-02-17,2008-02-09,-62.178873,-58.907921,54.0,8,-0.01600,18.954,-0.20554
...,...,...,...,...,...,...,...,...,...,...
1805,9N,2012-02-04,2012-01-20,-62.166223,-58.883543,261.0,15,0.79370,87.260,-0.07890
1806,9N,2012-02-19,2012-02-04,-62.166223,-58.883543,261.0,15,0.71200,79.370,-0.08170
1807,9N,2012-03-04,2012-02-19,-62.166223,-58.883543,261.0,14,0.80920,71.200,0.09720
1808,9N,2012-03-17,2012-03-04,-62.166223,-58.883543,261.0,13,0.59040,80.920,-0.21880


In [8]:
smb_coll_df1 = df_s[['Stake', 'Date', 'Date1', 'Latitude', 'Longitude', 
                     'Elevation', 'Days','SMB_diff']]

In [9]:
smb_coll_df1.rename(columns={'SMB_diff':'SMB'}, inplace=True)

In [10]:
smb_coll_df1.to_csv('../data/SMB_bellin_diff.csv', index=False, sep='\t')